This is the work-counting version of makemore for trigrams

Load all the words.

In [134]:
words = []
with open("names.txt") as file:
    words = file.read().splitlines()
words[:3]

['emma', 'olivia', 'ava']

Encode all the words with integer values for the letters.

In [135]:
chars = ".abcdefghijklmnopqrstuvwxyz"
char_to_index = {}
index = 0
for char in list(chars):
    char_to_index[char] = index
    index += 1

index_to_char = {index:char for char,index in char_to_index.items()}
all_words_encoded = []
for word in words:
    letters = [".", "."] + list(word) + ["."]
    # letters = ["."] + list(word) + ["."]
    all_words_encoded.append([char_to_index[letter] for letter in letters])

def btoi(chars):
    if len(chars) != 2:
        raise Exception("bigram_to_index accepts only strings of two characters where both are either letters or \".\"")
    return char_to_index[chars[0]] * 27 + char_to_index[chars[1]]
#"".join([index_to_char[char] for char in encoded_words[0]])

Split all the words into trigrams. Represent the input (the first two characters) as the second + 27 * the first. Count the # of occurrences of each and index by the bigram.

In [136]:
import torch
counts = torch.zeros(27*27, 27, dtype=float)
counts += .01
print(counts.shape)

n_observations = 0
for word_encoded in all_words_encoded[:]:
    for index1, index2, index3 in zip(word_encoded, word_encoded[1:], word_encoded[2:]):
        x = (index1 * 27) + index2
        y = index3
        counts[x,y] += 1
        n_observations += 1

counts[728]

torch.Size([729, 27])


tensor([4.0100e+00, 1.3010e+01, 1.0000e-02, 1.0000e-02, 1.0000e-02, 2.0100e+00,
        1.0000e-02, 1.0000e-02, 1.0000e-02, 8.0100e+00, 1.0000e-02, 1.0000e-02,
        5.0100e+00, 4.0100e+00, 1.0000e-02, 1.0100e+00, 1.0000e-02, 1.0000e-02,
        1.0000e-02, 1.0000e-02, 1.0000e-02, 1.0100e+00, 1.0000e-02, 1.0000e-02,
        1.0000e-02, 7.0100e+00, 1.0000e-02], dtype=torch.float64)

Normalize the counts for each bigram    

In [137]:
probs = counts / counts.sum(1, keepdim=True)

In [138]:
probs[245, 0]

tensor(0.2268, dtype=torch.float64)

In [141]:
for i in range(20):
  output = []
  bigram_index = 0 # the bigram representing two word-starts characters.
  
  while True:
    next_index = torch.multinomial(probs[bigram_index], 1, replacement=True).item()
    next_char = index_to_char[next_index]
    output.append(next_char)
    if next_char == ".":
      break
    bigram_index = (bigram_index % 27) * 27 + next_index
  print("".join(output))

ola.
cliz.
rhylileimbron.
mionnee.
prinie.
drition.
ryddica.
cri.
cley.
zubliangen.
avie.
joysi.
arliegri.
luwavlamehah.
ozelyn.
ranaelykenzelilen.
ryatti.
alah.
bryahnianal.
ell.


In [142]:
bii = btoi("yd")
for i in range(len(probs[bii])):
    print(index_to_char[i], probs[bii][i])

. tensor(0.0331, dtype=torch.float64)
a tensor(0.1653, dtype=torch.float64)
b tensor(3.6728e-05, dtype=torch.float64)
c tensor(3.6728e-05, dtype=torch.float64)
d tensor(0.0037, dtype=torch.float64)
e tensor(0.4444, dtype=torch.float64)
f tensor(3.6728e-05, dtype=torch.float64)
g tensor(0.0037, dtype=torch.float64)
h tensor(3.6728e-05, dtype=torch.float64)
i tensor(0.1506, dtype=torch.float64)
j tensor(3.6728e-05, dtype=torch.float64)
k tensor(3.6728e-05, dtype=torch.float64)
l tensor(3.6728e-05, dtype=torch.float64)
m tensor(3.6728e-05, dtype=torch.float64)
n tensor(0.0515, dtype=torch.float64)
o tensor(0.0698, dtype=torch.float64)
p tensor(3.6728e-05, dtype=torch.float64)
q tensor(3.6728e-05, dtype=torch.float64)
r tensor(0.0257, dtype=torch.float64)
s tensor(3.6728e-05, dtype=torch.float64)
t tensor(3.6728e-05, dtype=torch.float64)
u tensor(0.0037, dtype=torch.float64)
v tensor(3.6728e-05, dtype=torch.float64)
w tensor(3.6728e-05, dtype=torch.float64)
x tensor(3.6728e-05, dtype=torch